# DAMLAS - Machine Learning At Scale
## Assignment - HW3
Data Analytics and Machine Learning at Scale
Target, Bangalore

---
__Name:__  *Your Name Goes Here*   
__Class:__ DAMLAS (Section *Your Section Goes Here*, e.g., Fall 2016)     
__Email:__  *Your Target Email User Goes Here*@Target.com     
__Week:__   01

# Table of Contents <a name="TOC"></a> 

1.  [HW Introduction](#1)   
2.  [HW References](#2)
3.  [HW  Problems](#3)   
    1.0.  [HW3.1](#1.1)   
    1.2.  [HW3.2](#1.2)   
    1.3.  [HW3.3](#1.3)    
    1.4.  [HW3.4](#1.4)    
    


<a name="1">
# 1 Instructions
[Back to Table of Contents](#TOC)
* Homework submissions are due by Thursday, 08/30/2016 at 10:30AM.


* Prepare a single Jupyter notebook (not a requirment), please include questions, and question numbers in the questions and in the responses.
Submit your homework notebook via the following form:

   + [Submission Link - Google Form](http://goo.gl/forms/er3OFr5eCMWDngB72)


### Documents:
* IPython Notebook, published and viewable online.
* PDF export of IPython Notebook.

<a name="2">
# 2 Useful References
[Back to Table of Contents](#TOC)

* [Total Sort Notebook](http://nbviewer.jupyter.org/urls/dl.dropbox.com/s/44idk0xz2wyu45l/total-sort-guide.ipynb) 
* [Hadoop Primer](https://www.dropbox.com/s/g84vzatuyalhe9y/Lecture%203-Hadoop-Intro-2016-06-13.pdf?dl=0)
* [Word Count example using Hadoop Streaming and  Python](http://www.michael-noll.com/tutorials/writing-an-hadoop-mapreduce-program-in-python/
)
---


In [ ]:
pip install mrjob

In [ ]:
export PATH="/home/username/anaconda/bin:$PATH"

## locate the hadoop streaming jar

In [1]:
! find / -name "hadoop*streaming*" -print 1> tmp.hadoopJar 2> /dev/null

^C


In [2]:
!cat tmp.hadoopJar

/Users/jshanahan/Google Drive/JShanahan/Publications/Conferences/STRATA-2012/tutorial_large_scale_mining/hadoop-0.20.2/contrib/streaming/hadoop-0.20.2-streaming.jar
/Users/jshanahan/Publications/Conferences/STRATA-2012/tutorial_large_scale_mining/hadoop-0.20.2/contrib/streaming/hadoop-0.20.2-streaming.jar
/usr/local/Cellar/hadoop/2.6.0/libexec/share/doc/hadoop/hadoop-streaming
/usr/local/Cellar/hadoop/2.6.0/libexec/share/hadoop/tools/lib/hadoop-streaming-2.6.0.jar
/usr/local/Cellar/hadoop/2.6.0/libexec/share/hadoop/tools/sources/hadoop-streaming-2.6.0-sources.jar
/usr/local/Cellar/hadoop/2.6.0/libexec/share/hadoop/tools/sources/hadoop-streaming-2.6.0-test-sources.jar


In [15]:
!ls -l /usr/lib/hadoop-0.20-mapreduce/contrib/streaming/hadoop-streaming-2.6.0-mr1-cdh5.8.0.jar



-rw-r--r-- 1 root root 107707 Jun 16 13:00 /usr/lib/hadoop-0.20-mapreduce/contrib/streaming/hadoop-streaming-2.6.0-mr1-cdh5.8.0.jar


In [5]:
!mkdir WordCount

In [5]:
%%writefile WordCount/mapper.py
#!/usr/bin/env python

import sys
#sys.stderr.write("reporter:counter:Tokens,Total,1") # NOTE missing the carriage return so wont work
sys.stderr.write("reporter:counter:Mapper Counters,Calls,1\n")
sys.stderr.write("reporter:status:processing my message...how are  you\n")

for line in sys.stdin:
    for word in line.split():
        print '%s\t%s' % (word, 1)


Writing WordCount/mapper.py


In [6]:
%%writefile WordCount/reducer.py
#!/usr/bin/env python

import sys

cur_key = None
cur_count = 0
sys.stderr.write("reporter:counter:Reducer Counters,Calls,1\n")
for line in sys.stdin:
    key, value = line.split()
    if key == cur_key:
        cur_count += int(value)
    else:
        if cur_key:
            print '%s\t%s' % (cur_key, cur_count)
        cur_key = key
        cur_count = int(value)

print '%s\t%s' % (cur_key, cur_count)


Writing WordCount/reducer.py


In [9]:
!chmod a+x WordCount/mapper.py

In [10]:
!chmod a+x WordCount/reducer.py

In [7]:
!echo "foo foo quux labs foo bar quux" | WordCount/mapper.py 

/bin/sh: WordCount/mapper.py: Permission denied


In [11]:
!echo "foo foo quux labs foo bar quux" | WordCount/mapper.py | sort -k1,1 | WordCount/reducer.py| sort -k2,2nr 

reporter:counter:Mapper Counters,Calls,1
reporter:status:processing my message...how are you
reporter:counter:Reducer Counters,Calls,1
foo	3
quux	2
bar	1
labs	1


In [14]:
!hdfs dfs -ls

In [23]:
%%writefile testWordCountInput.txt
hello this is Jimi
jimi who Jimi three Jimi 
Hello
hello

Overwriting testWordCountInput.txt


In [22]:
!cd WordCount/

In [25]:
!hdfs dfs -rm testWordCountInput.txt 
!hdfs dfs -copyFromLocal testWordCountInput.txt 
!hdfs dfs -rm -r wordcount-output
#usr/local/Cellar/hadoop/2.6.0/libexec/share/hadoop/tools/lib
dataDir = "/Users/jshanahan/Dropbox/lectures-uc-berkeley-ml-class-2015/Notebooks/WordCount"

!hadoop jar /usr/lib/hadoop-0.20-mapreduce/contrib/streaming/hadoop-streaming-2.6.0-mr1-cdh5.8.0.jar \
   -mapper /home/cloudera/WordCount/mapper.py \
   -reducer /home/cloudera/WordCount/reducer.py \
   -combiner /home/cloudera/WordCount/reducer.py \
   -input testWordCountInput.txt \
   -output wordcount-output  \
   -numReduceTasks 3
   #--D mapreduce.job.reduces=2  depecated
#-input historical_tours.txt  file on Hadoop


#output directory on Hadoop 

Deleted testWordCountInput.txt
Deleted wordcount-output
packageJobJar: [] [/usr/lib/hadoop-mapreduce/hadoop-streaming-2.6.0-cdh5.8.0.jar] /tmp/streamjob7511691146345687632.jar tmpDir=null
16/08/26 20:46:58 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
16/08/26 20:46:59 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
16/08/26 20:47:00 WARN hdfs.DFSClient: Caught exception 
java.lang.InterruptedException
	at java.lang.Object.wait(Native Method)
	at java.lang.Thread.join(Thread.java:1281)
	at java.lang.Thread.join(Thread.java:1355)
	at org.apache.hadoop.hdfs.DFSOutputStream$DataStreamer.closeResponder(DFSOutputStream.java:862)
	at org.apache.hadoop.hdfs.DFSOutputStream$DataStreamer.endBlock(DFSOutputStream.java:600)
	at org.apache.hadoop.hdfs.DFSOutputStream$DataStreamer.run(DFSOutputStream.java:789)
16/08/26 20:47:00 INFO mapred.FileInputFormat: Total input paths to process : 1
16/08/26 20:47:00 WARN hdfs.DFSClient: Caught exception 
java.lang.In

In [26]:
#have a look at the input
!echo  "\n---------------------------\n"
!hdfs dfs -cat testWordCountInput.txt
!echo  "\n---------------------------\n"
# Wordcount output
!hdfs dfs -cat wordcount-output/part-0000*

\n---------------------------\n
hello this is Jimi
jimi who Jimi three Jimi 
Hello
hello\n---------------------------\n
Hello	1
jimi	1
this	1
three	1
Jimi	3
hello	2
is	1
who	1


# 3.  HW3  <a name="3"></a>
[Back to Table of Contents](#TOC)

* Dont forget to create a new subdirectory for this homework (on your local machine/virtual machine, and on HDFS)
---
## 3.  HW3.1  <a name="1.1"></a>
[Back to Table of Contents](#TOC)

In Hadoop streaming write code to determing the top-10 most frequent  using  words present in the text version of Alice’s Adventures in Wonderland. (You can obtain a free plain text version of the book, along with many others, from [here](http://www.gutenberg.org.). First use one reducer and report your result. Then develop a solution when you use 3 reducers? 


* Please leverage the following notebook for this HW problem: [Total Sort Notebook](http://nbviewer.jupyter.org/urls/dl.dropbox.com/s/44idk0xz2wyu45l/total-sort-guide.ipynb)

For your result please  create a text file named __alice_sorted_words.txt__ containing an  listing of all the words, and the number of times each occurs, in the text version of Alice’s Adventures in Wonderland in the following sort order: WordCount descreasing, and where words that have the same count, they be ordered in alphabetical increasing. (You can obtain a free plain text version of the book, along with many others, from here The first 10 lines of your output file should look something like this:

In [ ]:
!curl 'http://www.gutenberg.org/cache/epub/11/pg11.txt' -o alicesTExtFilename.txt

## 3.  HW3.2  <a name="1.2"></a>
[Back to Table of Contents](#TOC)

The data file at http://www.metoffice.gov.uk/climate/uk/stationdata/heathrowdata.txt (mirrored here) is part of a set of text files cataloguing temperatures collected in different cities of the UK. The one for the URL given is for Heathrow (London Airport). Typically each city is held in one data file. You can access the whole set from [here](http://www.metoffice.gov.uk/climate/uk/stationdata/).
Write a MapReduce application with a mapper.py and reduce.py that uses only the text file for Heathrow as input and that outputs all records in the following order: date increasing and temperature decreasing. 
